1. 参考课堂案例，使用指定的数据集，编写代码实现ner模型训练和推流。
https://huggingface.co/datasets/doushabao4766/msra_ner_k_V3

In [1]:
from datasets import load_dataset

try:
    # 直接通过数据集路径加载
    dataset = load_dataset("doushabao4766/msra_ner_k_V3")
    print(dataset)
except Exception as e:
    print(f"加载失败，错误信息：{e}")
    # 尝试指定缓存路径或重新下载
    dataset = load_dataset("doushabao4766/msra_ner_k_V3", download_mode="force_redownload")

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-42717a92413393f9.parquet:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

(…)-00000-of-00001-8899cab5fdab45bc.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})


In [2]:
from datasets import load_dataset

# 加载数据集
dataset = load_dataset("doushabao4766/msra_ner_k_V3")

# 查看数据集结构
print(dataset)

# 获取标签名称映射
if 'train' in dataset:  # 检查训练集是否存在
    # 获取标签特征
    ner_feature = dataset['train'].features['ner_tags']
    
    # 打印标签名称列表
    print("\nNER标签对应的实体类型:")
    for idx, label in enumerate(ner_feature.feature.names):
        print(f"{idx}: {label}")
else:
    print("数据集中不包含训练集。")

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})

NER标签对应的实体类型:
0: O
1: B-PER
2: I-PER
3: B-ORG
4: I-ORG
5: B-LOC
6: I-LOC


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
train_=dataset['train']
train_

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'knowledge'],
    num_rows: 45001
})

In [5]:
from transformers import AutoModelForTokenClassification,AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [6]:
def data_input_proc(item):
    ##输入文本转换模型输入token索引
    text = ["".join(val) for val in item['tokens']]
    input_data=tokenizer(text,truncation=True,add_special_tokens=False,max_length=512)
    adjust_labels=[]
    ##上一步骤生成ner_tag中索引和token对齐
    for k in range(len(input_data['input_ids'])):

        word_ids=input_data.word_ids(k)
        tags=item['ner_tags'][k]
        
        adjusted_label_ids=[]
        i,prev_wid=-1,-1
        for wid in word_ids:
            if (wid!=prev_wid):
                i+=1
                prev_wid=wid
            adjusted_label_ids.append(tags[i])
        adjust_labels.append(adjusted_label_ids)
    input_data['labels']=adjust_labels
    return input_data 

ds2=dataset.map(data_input_proc,batched=True) #batchted 每次传入自定义方法样板数量

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [7]:
for item in ds2['train']:
    print(item)
    break

{'id': '0', 'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'knowledge': '', 'input_ids': [2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [8]:
##记录转换为pytorch
ds2.set_format('torch',columns=['input_ids','token_type_ids','attention_mask','labels'])

####模型训练

In [9]:
for item in ds2['train']:
    print(item)
    break

{'input_ids': tensor([2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636,  674, 1036, 4997,
        2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768,
        7599, 3198, 8024,  791, 1921, 3300, 3119, 5966,  817,  966, 4638,  741,
         872, 3766,  743, 8024, 3209, 3189, 2218, 1373,  872, 2637,  679, 2496,
        1159, 8013]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]), 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])}


In [10]:
from transformers import TrainingArguments,Trainer

args=TrainingArguments(
    output_dir="ner_train"  ##模型训练工作目录
    ,num_train_epochs=2
    ,save_safetensors=False  ##特定格式的文件,是否保存为torch.load格式
    ,per_device_train_batch_size=32 #训练批次
    ,per_device_eval_batch_size=32
    ,report_to='tensorboard' #模型输出记录
     ,eval_strategy='epoch' # No  steps epoch 
)

2025-06-01 08:52:38.437067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748767958.618908      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748767958.671026      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
# 手动收集所有出现过的标签 ID
all_tag_ids = set()
for example in dataset['train']:
    all_tag_ids.update(example['ner_tags'])

# 将 ID 转换为标签名称（如果已知映射关系）
id_to_tag = dataset['train'].features['ner_tags'].feature.names
tag_names = [id_to_tag[i] for i in sorted(all_tag_ids)]

# 创建映射
id2lbl = {i: tag for i, tag in enumerate(tag_names)}
lbl2id = {tag: i for i, tag in enumerate(tag_names)}

In [12]:
print(tag_names)
print(id2lbl)
print(lbl2id)
print(tag_names[1])

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}
B-PER


In [13]:
###Model
from transformers import AutoModelForTokenClassification
model=AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                     num_labels=7,
                                                     id2label=id2lbl,
                                                     label2id=lbl2id)
model

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [14]:
!pip install evaluate
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=05f61b837c675f3166f7be36c39f15d574ca0ee5a5015510f9410e3ebcc1f420
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [15]:
from transformers import DataCollatorForTokenClassification
import evaluate
import seqeval

In [16]:
##metric方法
def compute_metric(result):
    ##获取评估对象
    seqeval=evaluate.load('seqeval')
    predicts,labels=result
    predicts=np.argmax(predicts,axis=2)
    ###获取评估对象
    seqeval=evaluate.load('seqeval')
    ###准备评估数据
    predicts=[[tag_names[p] for p,l in zip(ps,ls) if l!=-100]
                    for ps,ls in zip(predicts,labels)]
    labels=[[tag_names[l] for p,l in zip(ps,ls) if l!=-100]
                    for ps,ls in zip(predicts,labels) ]
    results=seqeval.compute(predictions=predicts,references=labels)
    return results

In [17]:
##Trainer

data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer
                                                ,padding=True)
trainer=Trainer(
    model,
    args,
    train_dataset=ds2['train'],
    eval_dataset=ds2['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
    
)

In [18]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.108100,0.046691,"{'precision': 0.865518504946867, 'recall': 0.8296452406041447, 'f1': 0.8472022955523674, 'number': 2847}","{'precision': 0.7580174927113703, 'recall': 0.7884761182714177, 'f1': 0.7729468599033816, 'number': 1319}","{'precision': 0.8762541806020067, 'recall': 0.8692767086927671, 'f1': 0.8727514990006662, 'number': 1507}",0.842030,0.830601,0.836277,0.984538
2,0.043900,0.044094,"{'precision': 0.870456998920475, 'recall': 0.84966631541974, 'f1': 0.8599360113757554, 'number': 2847}","{'precision': 0.7828282828282829, 'recall': 0.8225928733889311, 'f1': 0.8022181146025877, 'number': 1319}","{'precision': 0.8942498347653668, 'recall': 0.8978102189781022, 'f1': 0.8960264900662253, 'number': 1507}",0.855407,0.856161,0.855784,0.986470


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Trainer is attempting to log a value of "{'precision': 0.865518504946867, 'recall': 0.8296452406041447, 'f1': 0.8472022955523674, 'number': 2847}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7580174927113703, 'recall': 0.7884761182714177, 'f1': 0.7729468599033816, 'number': 1319}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8762541806020067, 'recall': 0.8692767086927671, 'f1': 0.8727514990006662, 'number': 1507}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to ga

TrainOutput(global_step=1408, training_loss=0.06329780681566759, metrics={'train_runtime': 1527.2386, 'train_samples_per_second': 58.931, 'train_steps_per_second': 0.922, 'total_flos': 7795678239418674.0, 'train_loss': 0.06329780681566759, 'epoch': 2.0})

####模型推理

In [19]:
result=trainer.predict(ds2['test'])


In [31]:
print(dataset['test'][10]['tokens'])
print(ds2['test'][10]['labels'])
# print('predict:',result.label_ids[10] )
pred=[l  for l in result.label_ids[10]  if l!=-100]
print(pred)                 

['在', '跨', '世', '纪', '的', '征', '途', '上', '，', '在', '中', '国', '共', '产', '党', '领', '导', '下', '，', '我', '们', '要', '努', '力', '实', '现', '包', '括', '各', '民', '主', '党', '派', '、', '各', '人', '民', '团', '体', '、', '无', '党', '派', '人', '士', '在', '内', '的', '全', '体', '中', '国', '人', '民', '的', '大', '团', '结', '，', '实', '现', '包', '括', '大', '陆', '同', '胞', '、', '台', '港', '澳', '同', '胞', '和', '海', '外', '侨', '胞', '在', '内', '的', '所', '有', '爱', '国', '的', '中', '华', '儿', '女', '的', '大', '团', '结', '，', '从', '而', '战', '胜', '各', '种', '艰', '难', '险', '阻', '，', '实', '现', '跨', '世', '纪', '的', '宏', '伟', '蓝', '图', '。']
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 5, 5, 5, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
[0, 0, 0, 0, 0, 0, 0, 

In [34]:
ds2['test']

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3443
})

In [63]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 加载预训练模型和分词器
##导入模型分词器 
# model_name = "dslim/bert-base-NER"  # 预训练NER模型
# 也可以使用其他模型，例如: "bert-base-chinese" 或 "xlm-roberta-large-finetuned-conll03-english"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name).to(device)

# 定义标签映射（根据具体模型可能需要调整）
# id2label = model.config.id2label
# 
# 创建NER推理管道
ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # 合并子词预测
    device=device.index if device.type == "cuda" else -1
)

def predict_entities(text: str) -> list:
    """
    对输入文本进行NER预测
    
    Args:
        text: 输入文本
    
    Returns:
        识别出的实体列表，每个实体包含文本、类型、起始和结束位置
    """
    # 使用管道进行预测
    entities = ner_pipeline(text)
    
    # 处理预测结果
    results = []
    for entity in entities:
        results.append({
            "entity": entity["entity_group"],
            "content": entity["word"].replace(" ","")
        })
    
    return results




Device set to use cuda:0


使用设备: cuda


In [66]:
# 示例用法

sample_texts = [
    "豆包在上海的复旦大学参加学术会议",
    "双方确定了今后发展中美关系的指导方针",
    "习近平总书记在上海考察调研。",  # 中文示例
     "张三今天去了北京故宫博物院参观。",
    "阿里巴巴集团成立于1999年，总部位于杭州。",
    "王小明在清华大学购买了华为手机。",
    "上海复旦大学附属中山医院位于徐汇区。",
    "鲁迅的《呐喊》是中国现代文学经典。",
    "2023年7月我在纽约参加学术会议。",
    "腾讯CEO马化腾出席了深圳科技大会。",
    "从北京到上海需要经过济南和南京。",
    "科学家李四在中科院物理研究所发表论文。",
    "《哈利·波特》作者J.K.罗琳居住在英国伦敦。"
]

# 对每个测试文本进行预测
for text in sample_texts:
    entities = predict_entities(text)
    print("输入：",text)
    print("输出：",entities)
    print()


输入： 豆包在上海的复旦大学参加学术会议
输出： [{'entity': 'PER', 'content': '豆包'}, {'entity': 'LOC', 'content': '上海的'}, {'entity': 'ORG', 'content': '复旦大学'}]

输入： 双方确定了今后发展中美关系的指导方针
输出： [{'entity': 'LOC', 'content': '中'}, {'entity': 'LOC', 'content': '美'}]

输入： 习近平总书记在上海考察调研。
输出： [{'entity': 'PER', 'content': '习近平'}, {'entity': 'LOC', 'content': '上海'}]

输入： 张三今天去了北京故宫博物院参观。
输出： [{'entity': 'PER', 'content': '张三'}, {'entity': 'ORG', 'content': '北'}, {'entity': 'LOC', 'content': '京'}, {'entity': 'ORG', 'content': '故宫博物院'}]

输入： 阿里巴巴集团成立于1999年，总部位于杭州。
输出： [{'entity': 'ORG', 'content': '阿里巴巴集团'}, {'entity': 'LOC', 'content': '杭州'}]

输入： 王小明在清华大学购买了华为手机。
输出： [{'entity': 'PER', 'content': '王小明'}, {'entity': 'ORG', 'content': '清华大学'}, {'entity': 'ORG', 'content': '华为手'}]

输入： 上海复旦大学附属中山医院位于徐汇区。
输出： [{'entity': 'ORG', 'content': '上海复旦大学附属中山医院'}, {'entity': 'LOC', 'content': '徐汇区'}]

输入： 鲁迅的《呐喊》是中国现代文学经典。
输出： [{'entity': 'PER', 'content': '鲁迅'}, {'entity': 'LOC', 'content': '中国'}]

输入： 2023年7月我在纽约参加学术会议。
输出： [{'ent